In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"C:\Users\lenovo\Downloads\train.csv\train.csv")

In [3]:
df.shape

(404290, 6)

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
new_df = df.sample(30000)

In [ ]:
# new_df = df.sample(30000)
# It randomly selects 30,000 rows — but it gives different results every time you run the code (because it's random!).


# Now if you want to get the same 30,000 rows every time you run the code (e.g., for reproducibility in machine learning or debugging), you set the random_state:

# new_df = df.sample(30000, random_state=42)

In [6]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [7]:
new_df.duplicated().sum()

0

In [8]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
262116,What does chronological order mean? What are s...,"What does ""fragile"" mean? What are some examples?"
170276,Which is the best site to download movies?,What are the best site for HD movies downloading?
255453,Which is the best Kannada movie according to you?,Why do Japanese men brag about having small pe...
270698,What does it take to be a top writer on Quora?,How many views and answers are required to bec...
283294,Do psychopaths feel shame or embarrassment if ...,How would a psychopath react if they felt like...


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
# Merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])
# Now questions(variable) is a list which have 8 lakh questions

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [13]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [14]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
262116,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
170276,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
255453,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
270698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
283294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33878,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50175,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
327053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
273257,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Adding a new column in temp_df
temp_df['is_duplicate'] = new_df['is_duplicate']

In [16]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
262116,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
170276,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
255453,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
270698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
283294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
accuracy_score(y_test,y_pred)

0.7401666666666666

In [21]:
pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 1.3/150.0 MB 11.2 MB/s eta 0:00:14
   - -------------------------------------- 4.2/150.0 MB 12.6 MB/s eta 0:00:12
   - -------------------------------------- 6.8/150.0 MB 12.7 MB/s eta 0:00:12
   -- ------------------------------------- 10.2/150.0 MB 13.6 MB/s eta 0:00:11
   --- ------------------------------------ 13.4/150.0 MB 14.0 MB/s eta 0:00:10
   ---- ----------------------------------- 16.5/150.0 MB 13.9 MB/s eta 0:00:10
   ----- ---------------------------------- 19.4/150.0 MB 13.9 MB/s eta 0:00:10
   ------ --------------------------------- 22.5/150.0 MB 13.8 MB/s eta 0:00:10
   ------ --------------------------------- 25.2/150.0 MB 13.9 MB/s eta 0:00:10
   ------- -------------------------------- 28.0/150.0 MB 13.9 MB/s eta 0:00:09
   -------- ------------------------------- 30.9/150.0 MB 13.9 MB/s eta 0:00:09
   --------- ------------------------------ 33.8/150

In [23]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(x_train,y_train)
y_pred = xgb.predict(x_test)
accuracy_score(y_test,y_pred)

0.7248333333333333